### Acquire the telco data

In [1]:
import env
import numpy as np
import pandas as pd
import acquire

telco_query = """
SELECT * FROM customers 
JOIN contract_types ON contract_types.contract_type_id = customers.contract_type_id
JOIN internet_service_types ON internet_service_types.internet_service_type_id = customers.internet_service_type_id
JOIN payment_types ON payment_types.payment_type_id = customers.payment_type_id;
"""

telco_url = f"mysql+pymysql://{env.user}:{env.password}@{env.host}/telco_churn"

In [2]:
#customers = pd.read_sql(telco_query, telco_url)
#customers.head()

In [3]:
#For testing
customers = acquire.get_telco_data()
customers.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,2,90.45,5957.9,No,3,Two year,1,DSL,2,Mailed check
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,4,45.20,2460.55,No,3,Two year,1,DSL,4,Credit card (automatic)
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,3,45.05,2560.1,No,3,Two year,1,DSL,3,Bank transfer (automatic)
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,4,39.40,825.4,No,3,Two year,1,DSL,4,Credit card (automatic)
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,3,85.15,6316.2,No,3,Two year,1,DSL,3,Bank transfer (automatic)


In [4]:
customers.shape

(7043, 27)

### Prepare the data

In [5]:
#Check for duplicates
customers.drop_duplicates(inplace = True)
customers.shape

(7043, 27)

In [6]:
#Check for null values
customers.isna().sum()

customer_id                   0
gender                        0
senior_citizen                0
partner                       0
dependents                    0
tenure                        0
phone_service                 0
multiple_lines                0
internet_service_type_id      0
online_security               0
online_backup                 0
device_protection             0
tech_support                  0
streaming_tv                  0
streaming_movies              0
contract_type_id              0
paperless_billing             0
payment_type_id               0
monthly_charges               0
total_charges                 0
churn                         0
contract_type_id.1            0
contract_type                 0
internet_service_type_id.1    0
internet_service_type         0
payment_type_id.1             0
payment_type                  0
dtype: int64

In [7]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customer_id                 7043 non-null   object 
 1   gender                      7043 non-null   object 
 2   senior_citizen              7043 non-null   int64  
 3   partner                     7043 non-null   object 
 4   dependents                  7043 non-null   object 
 5   tenure                      7043 non-null   int64  
 6   phone_service               7043 non-null   object 
 7   multiple_lines              7043 non-null   object 
 8   internet_service_type_id    7043 non-null   int64  
 9   online_security             7043 non-null   object 
 10  online_backup               7043 non-null   object 
 11  device_protection           7043 non-null   object 
 12  tech_support                7043 non-null   object 
 13  streaming_tv                7043 

In [8]:
#Nothing is missing... Didn't expect that
#Convert total_charges to float
customers.total_charges = customers.total_charges.str.strip()
customers.total_charges = customers.total_charges.str.replace('[$,]','')
customers.total_charges = pd.to_numeric(customers['total_charges'])
#Select the categorical columns that need dummy variables (ignoring passenger_id)
cat_cols = customers.select_dtypes('object').columns[1:]

<ipython-input-8-ea7f2dbf3214>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  customers.total_charges = customers.total_charges.str.replace('[$,]','')


In [9]:
#Create dummy variables for categorical variables
dummy_df = pd.get_dummies(customers[cat_cols], dummy_na = False, drop_first = True)
dummy_df.head()

,gender_Male,partner_Yes,dependents_Yes,phone_service_Yes,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,online_security_Yes,online_backup_No internet service,online_backup_Yes,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,1,1,1,0,1,0,1,0,1,...,1,1,0,0,1,0,0,0,0,1
1,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
3,1,1,1,0,1,0,0,1,0,0,...,0,1,0,0,1,0,0,1,0,0
4,1,1,0,1,0,1,0,0,0,1,...,1,1,0,0,1,0,0,0,0,0


In [10]:
customers = pd.concat([customers, dummy_df], axis = 1)
customers.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,1,1,0,0,1,0,0,0,0,1
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,0,0,0,0,1,0,0,1,0,0
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,0,0,0,0,1,0,0,0,0,0
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,0,1,0,0,1,0,0,1,0,0
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,1,1,0,0,1,0,0,0,0,0


In [11]:
customers.drop(columns = cat_cols, inplace = True)
customers.head()

,customer_id,senior_citizen,tenure,internet_service_type_id,contract_type_id,payment_type_id,monthly_charges,total_charges,contract_type_id.1,internet_service_type_id.1,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0016-QLJIS,0,65,1,3,2,90.45,5957.90,3,1,...,1,1,0,0,1,0,0,0,0,1
1,0017-DINOC,0,54,1,3,4,45.20,2460.55,3,1,...,0,0,0,0,1,0,0,1,0,0
2,0019-GFNTW,0,56,1,3,3,45.05,2560.10,3,1,...,0,0,0,0,1,0,0,0,0,0
3,0056-EPFBG,0,20,1,3,4,39.40,825.40,3,1,...,0,1,0,0,1,0,0,1,0,0
4,0078-XZMHT,0,72,1,3,3,85.15,6316.20,3,1,...,1,1,0,0,1,0,0,0,0,0


In [12]:
customers.drop('customer_id', axis = 1, inplace = True)
customers.head()

,senior_citizen,tenure,internet_service_type_id,contract_type_id,payment_type_id,monthly_charges,total_charges,contract_type_id.1,internet_service_type_id.1,payment_type_id.1,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,65,1,3,2,90.45,5957.90,3,1,2,...,1,1,0,0,1,0,0,0,0,1
1,0,54,1,3,4,45.20,2460.55,3,1,4,...,0,0,0,0,1,0,0,1,0,0
2,0,56,1,3,3,45.05,2560.10,3,1,3,...,0,0,0,0,1,0,0,0,0,0
3,0,20,1,3,4,39.40,825.40,3,1,4,...,0,1,0,0,1,0,0,1,0,0
4,0,72,1,3,3,85.15,6316.20,3,1,3,...,1,1,0,0,1,0,0,0,0,0


In [13]:
customers.drop(columns = ['contract_type_id', 'contract_type_id.1', 'internet_service_type_id', 'internet_service_type_id.1', 'payment_type_id', 'payment_type_id.1'], inplace = True)
customers.head()

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,partner_Yes,dependents_Yes,phone_service_Yes,multiple_lines_No phone service,multiple_lines_Yes,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,65,90.45,5957.90,0,1,1,1,0,1,...,1,1,0,0,1,0,0,0,0,1
1,0,54,45.20,2460.55,1,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,0
2,0,56,45.05,2560.10,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0,20,39.40,825.40,1,1,1,0,1,0,...,0,1,0,0,1,0,0,1,0,0
4,0,72,85.15,6316.20,1,1,0,1,0,1,...,1,1,0,0,1,0,0,0,0,0


In [14]:
customers.drop('phone_service_Yes', axis = 1, inplace = True)
customers.head()

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,partner_Yes,dependents_Yes,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,65,90.45,5957.90,0,1,1,0,1,0,...,1,1,0,0,1,0,0,0,0,1
1,0,54,45.20,2460.55,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,56,45.05,2560.10,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,20,39.40,825.40,1,1,1,1,0,0,...,0,1,0,0,1,0,0,1,0,0
4,0,72,85.15,6316.20,1,1,0,0,1,0,...,1,1,0,0,1,0,0,0,0,0


In [15]:
#For testing
telco = acquire.get_telco_data()
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id.1,contract_type,internet_service_type_id.1,internet_service_type,payment_type_id.1,payment_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,2,90.45,5957.9,No,3,Two year,1,DSL,2,Mailed check
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,4,45.20,2460.55,No,3,Two year,1,DSL,4,Credit card (automatic)
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,3,45.05,2560.1,No,3,Two year,1,DSL,3,Bank transfer (automatic)
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,4,39.40,825.4,No,3,Two year,1,DSL,4,Credit card (automatic)
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,3,85.15,6316.2,No,3,Two year,1,DSL,3,Bank transfer (automatic)


In [17]:
import prepare
telco = prepare.prep_telco(telco)
telco.head()

/Users/johnathonsmith/codeup-data-science/classification-exercises/prepare.py:25: FutureWarning: The default value of regex will change from True to False in a future version.
  customers.total_charges = customers.total_charges.str.replace('[$,]','')


,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,partner_Yes,dependents_Yes,multiple_lines_No phone service,multiple_lines_Yes,online_security_No internet service,...,streaming_movies_Yes,paperless_billing_Yes,churn_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0,65,90.45,5957.90,0,1,1,0,1,0,...,1,1,0,0,1,0,0,0,0,1
1,0,54,45.20,2460.55,1,0,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,56,45.05,2560.10,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,20,39.40,825.40,1,1,1,1,0,0,...,0,1,0,0,1,0,0,1,0,0
4,0,72,85.15,6316.20,1,1,0,0,1,0,...,1,1,0,0,1,0,0,0,0,0


In [18]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   senior_citizen                         7043 non-null   int64  
 1   tenure                                 7043 non-null   int64  
 2   monthly_charges                        7043 non-null   float64
 3   total_charges                          7032 non-null   float64
 4   gender_Male                            7043 non-null   uint8  
 5   partner_Yes                            7043 non-null   uint8  
 6   dependents_Yes                         7043 non-null   uint8  
 7   multiple_lines_No phone service        7043 non-null   uint8  
 8   multiple_lines_Yes                     7043 non-null   uint8  
 9   online_security_No internet service    7043 non-null   uint8  
 10  online_security_Yes                    7043 non-null   uint8  
 11  onli